# 3장: 역할 할당 (역할 프롬프팅)

- [수업](#lesson)
- [연습문제](#exercises)
- [예시 플레이그라운드](#example-playground)

## 설정

다음 설정 셀을 실행하여 API 키를 로드하고 `get_completion` 헬퍼 함수를 설정합니다.

In [ ]:
# Import python's built-in regular expression library
import re
import boto3
from botocore.exceptions import ClientError
import json

# Import the hints module from the utils package
from utils import hints

# Retrieve the MODEL_NAME variable from the IPython store
%store -r modelId
%store -r region

bedrock_client = boto3.client(service_name='bedrock-runtime', region_name=region)

In [ ]:
def get_completion(prompt, system_prompt=None):
    inference_config = {
        "temperature": 0.0,
        "maxTokens": 500
    }
    converse_api_params = {
        "modelId": modelId,
        "messages": [{"role": "user", "content": [{"text": prompt}]}],
        "inferenceConfig": inference_config
    }
    if system_prompt:
        converse_api_params["system"] = [{"text": system_prompt}]
    try:
        response = bedrock_client.converse(**converse_api_params)
        text_content = response['output']['message']['content'][0]['text']
        return text_content

    except ClientError as err:
        message = err.response['Error']['Message']
        print(f"A client error occured: {message}")

---

## 수업

Claude가 당신이 말한 것 외에는 맥락을 모른다는 주제를 이어가면서, 때로는 **Claude에게 특정 역할을 수행하도록 프롬프트하는 것이 중요합니다(필요한 모든 맥락 포함)**. 이것을 역할 프롬프팅이라고 합니다. 역할 맥락이 자세할수록 더 좋습니다.

**Claude에게 역할을 부여하면 Claude의 성능이 향상될 수 있습니다.** 작문, 코딩, 요약 등 다양한 분야에서 그렇습니다. 마치 사람들에게 "~처럼 생각해봐"라고 말하면 도움이 되는 것과 같습니다. 역할 프롬프팅은 또한 Claude의 응답 스타일, 어조, 방식을 변화시킬 수 있습니다.

**참고:** 역할 프롬프팅은 시스템 프롬프트에서 이루어지거나 사용자 메시지 차례에서 이루어질 수 있습니다.

### 예시

아래 예시에서 볼 수 있듯이, 역할 프롬프팅 없이는 Claude가 스케이트보딩에 대한 단일 문장 관점을 요청받았을 때 **단순하고 스타일링되지 않은 답변**을 제공합니다.

하지만 Claude에게 고양이 역할을 부여하면, Claude의 관점이 변화하고 따라서 **Claude의 응답 어조, 스타일, 내용이 새로운 역할에 맞춰 적응**합니다.

**참고:** 사용할 수 있는 추가 기술은 **Claude에게 의도한 청중에 대한 맥락을 제공하는 것**입니다. 아래에서 "너는 고양이야"라고 프롬프트하는 것과 "너는 스케이트보더들 앞에서 말하는 고양이야"라고 프롬프트하는 것은 상당히 다른 응답을 이끌어낼 것입니다.

시스템 프롬프트에 역할 프롬프팅이 없는 프롬프트는 다음과 같습니다:

In [ ]:
# Prompt
PROMPT = "In one sentence, what do you think about skateboarding?"

# Print Claude's response
print(get_completion(PROMPT))

여기에 역할 프롬프팅이 있는 동일한 사용자 질문이 있습니다.

In [ ]:
# System prompt
SYSTEM_PROMPT = "You are a cat."

# Prompt
PROMPT = "In one sentence, what do you think about skateboarding?"

# Print Claude's response
print(get_completion(PROMPT, SYSTEM_PROMPT))

역할 프롬프팅을 사용하여 Claude가 특정 작문 스타일을 모방하거나 특정 어조로 말하게 하거나 답변의 복잡성을 안내할 수 있습니다. **역할 프롬프팅은 또한 Claude가 수학이나 논리 작업을 더 잘하도록 만들 수 있습니다.**

예를 들어, 아래 예시에서는 정답이 확실히 있습니다. 그런데 Claude가 오답을 내고 정보가 부족하다고 생각합니다:


In [ ]:
# Prompt
PROMPT = "Jack is looking at Anne. Anne is looking at George. Jack is married, George is not, and we don’t know if Anne is married. Is a married person looking at an unmarried person?"

# Print Claude's response
print(get_completion(PROMPT))

그렇다면 **Claude에게 논리 봇 역할을 부여**하면 어떻게 될까요?

이 새로운 역할 할당으로 인해 Claude가 정답을 맞힙니다. (비록 완전히 올바른 이유는 아닙니다)

In [ ]:
# System prompt
SYSTEM_PROMPT = "You are a logic bot designed to answer complex logic problems."

# Prompt
PROMPT = "Jack is looking at Anne. Anne is looking at George. Jack is married, George is not, and we don’t know if Anne is married. Is a married person looking at an unmarried person?"

# Print Claude's response
print(get_completion(PROMPT, SYSTEM_PROMPT))

**참고:** 이 과정을 통해 배우게 될 것은 유사한 결과를 얻기 위해 사용할 수 있는 **많은 프롬프트 엔지니어링 기술**이 있다는 것입니다. 어떤 기술을 사용할지는 여러분의 선택이며 선호에 달려 있습니다! 여러분만의 프롬프트 엔지니어링 스타일을 찾기 위해 **실험해 보시기를 권장**합니다.

위의 내용을 변경하지 않고 수업 프롬프트를 실험해 보고 싶다면 수업 노트북 하단으로 스크롤하여 [**예시 플레이그라운드**](#example-playground)를 방문하세요.

---

## 연습문제
- [연습문제 3.1 - 수학 문제 수정](#exercise-31---math-correction)

### 연습문제 3.1 - 수학 문제 수정
때로는 **Claude가 수학 문제를 잘 풀지 못할 수 있습니다**, 심지어 간단한 수학 문제도 말이죠. 아래에서 Claude는 두 번째 단계에서 명백한 산술 실수가 있음에도 불구하고 수학 문제가 올바르게 풀렸다고 잘못 평가합니다. Claude는 실제로 단계별로 진행하면서 실수를 잡아내지만, 전체 해답이 잘못되었다는 결론을 내리지는 못합니다.

`PROMPT` 및/또는 `SYSTEM_PROMPT`를 수정하여 Claude가 해답을 `잘못` 풀었다고 평가하도록 만드세요. 올바르게 풀었다고 평가하지 않도록 해주세요.


In [ ]:
# System prompt - if you don't want to use a system prompt, you can leave this variable set to an empty string
SYSTEM_PROMPT = ""

# Prompt
PROMPT = """Is this equation solved correctly below?

2x - 3 = 9
2x = 6
x = 3"""

# Get Claude's response
response = get_completion(PROMPT, SYSTEM_PROMPT)

# Function to grade exercise correctness
def grade_exercise(text):
    if "incorrect" in text or "not correct" in text.lower():
        return True
    else:
        return False

# Print Claude's response and the corresponding grade
print(response)
print("\n--------------------------- GRADING ---------------------------")
print("This exercise has been correctly solved:", grade_exercise(response))

❓ 힌트가 필요하다면 아래 셀을 실행하세요!

In [ ]:
print(hints.exercise_3_1_hint)

### 축하합니다!

지금까지의 모든 연습 문제를 해결했다면 다음 장으로 넘어갈 준비가 되었습니다. 행운이 있기를 바랍니다!

---

## 예시 플레이그라운드

여기는 이 수업에서 보여준 프롬프트 예시를 자유롭게 실험하고 프롬프트를 조정하여 Claude의 응답이 어떻게 달라지는지 확인할 수 있는 공간입니다.

In [ ]:
# Prompt
PROMPT = "In one sentence, what do you think about skateboarding?"

# Print Claude's response
print(get_completion(PROMPT))

In [ ]:
# System prompt
SYSTEM_PROMPT = "You are a cat."

# Prompt
PROMPT = "In one sentence, what do you think about skateboarding?"

# Print Claude's response
print(get_completion(PROMPT, SYSTEM_PROMPT))

In [ ]:
# Prompt
PROMPT = "Jack is looking at Anne. Anne is looking at George. Jack is married, George is not, and we don’t know if Anne is married. Is a married person looking at an unmarried person?"

# Print Claude's response
print(get_completion(PROMPT))

In [ ]:
# System prompt
SYSTEM_PROMPT = "You are a logic bot designed to answer complex logic problems."

# Prompt
PROMPT = "Jack is looking at Anne. Anne is looking at George. Jack is married, George is not, and we don’t know if Anne is married. Is a married person looking at an unmarried person?"

# Print Claude's response
print(get_completion(PROMPT, SYSTEM_PROMPT))